<a href="https://colab.research.google.com/github/Andy-Lewis-Sapner/EML_EcoCoders/blob/main/HW2/HW2_EcoCoders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Installations, Imports, Dataset Files And Constants

In [1]:
!pip install gradio pykrige -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 979.6/979.6 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 1.7 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.path import Path
from matplotlib.spines import Spine
from matplotlib.transforms import Affine2D
from mpl_toolkits.mplot3d import Axes3D
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from pykrige.ok import OrdinaryKriging
import gradio as gr

In [3]:
pca_data_file = "https://github.com/Andy-Lewis-Sapner/EML_EcoCoders/raw/main/Data/MeasurmentsAndIndividualsMeanValues.csv"
monitoring_reports_data_file = "https://github.com/Andy-Lewis-Sapner/EML_EcoCoders/raw/main/Data/KishonRiverMonitoringReports.xlsx"
biogis_data_file = "https://github.com/Andy-Lewis-Sapner/EML_EcoCoders/raw/main/Data/occurrences.csv"
meteo_data_file = "https://github.com/Andy-Lewis-Sapner/EML_EcoCoders/raw/main/Data/data_20222024.csv"

In [4]:
# Coordinates of monitoring stations (longitude, latitude)
station_coords = {
    'morad_reservoir_kfar_baruch': (35.193290585018204, 32.63799421872744),
    'quarry_station_jalama': (35.0988160806192, 32.72607628851692),
    'bridge_iri_yagur': (35.065171162350055, 32.75927717927157),
    'gypsum_mountain': (35.0691103835696, 32.777568354869544),
    'histadrut_bridge': (35.04857038220264, 32.79873407157874),
    'julius_simon_bridge': (35.03488553907516, 32.8016502054031),
    'stone_pier': (35.0285340685954, 32.808467653058855),
}

# App Logic

##PCA

Load data

In [5]:
eco_data = pd.read_csv(pca_data_file)
eco_data_dates = eco_data['year'].astype(int).astype(str) + '-' + eco_data['month'].astype(int).astype(str)
eco_data.insert(0, 'Date', eco_data_dates)
eco_data = eco_data.drop(columns=['year', 'month'])

Standardize the data

In [6]:
X = eco_data.iloc[:, 1:].values
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

Use PCA

In [7]:
# Apply PCA
pca = PCA()
pca_result = pca.fit_transform(X_scaled)

In [8]:
# Create DataFrame with PCA results
pca_df = pd.DataFrame(data=pca_result, columns=[f'PC{i+1}' for i in range(X.shape[1])])
pca_df['Date'] = eco_data_dates

In [9]:
# Get the loadings (correlation between variables and principal components)
loadings = pca.components_.T * np.sqrt(pca.explained_variance_)
loadings_df = pd.DataFrame(loadings, columns=[f'PC{i+1}' for i in range(X.shape[1])])
loadings_df['Variable'] = eco_data.columns[1:]

In [10]:
feature_names = eco_data.columns[1:]
correlation_matrix = eco_data.iloc[:, 1:].corr()
explained_variance = pca.explained_variance_ratio_ * 100
cumulative_variance = np.cumsum(explained_variance)

Scree Plot

In [11]:
def scree_plot():
  scree_plot_fig = plt.figure(figsize=(10, 6))
  plt.bar(range(1, len(explained_variance) + 1), explained_variance, alpha=0.7)
  plt.plot(range(1, len(explained_variance) + 1), cumulative_variance, 'ro-')
  plt.axhline(y=80, color='r', linestyle='--', alpha=0.5)  # Typical threshold line
  plt.xlabel('Principal Component')
  plt.ylabel('Explained Variance (%)')
  plt.title('Scree Plot with Cumulative Variance')
  plt.xticks(range(1, len(explained_variance) + 1))
  plt.legend(['Cumulative Explained Variance', 'Individual Explained Variance'])
  plt.grid(True, alpha=0.3)

  plt.close(scree_plot_fig)
  return scree_plot_fig

Biplot for PC1 and PC2

In [12]:
def biplot_pc1_pc2():
  biplot_fig = plt.figure(figsize=(12, 10))
  plt.scatter(pca_result[:, 0], pca_result[:, 1], s=100, alpha=0.7)
  for i, site in enumerate(eco_data_dates):
    plt.annotate(site, (pca_result[i, 0], pca_result[i, 1]),
                 textcoords="offset points", xytext=(0,10), ha='center')
  scaling_factor = 5  # Adjust this to scale the arrows appropriately
  for i, feature in enumerate(feature_names):
      plt.arrow(0, 0,
                loadings[i, 0] * scaling_factor,
                loadings[i, 1] * scaling_factor,
                color='r', alpha=0.7, head_width=0.2)
      plt.text(loadings[i, 0] * scaling_factor * 1.15,
              loadings[i, 1] * scaling_factor * 1.15,
              feature, color='g', ha='center', va='center')

  plt.xlabel(f'PC1 ({explained_variance[0]:.2f}%)')
  plt.ylabel(f'PC2 ({explained_variance[1]:.2f}%)')
  plt.title('PCA Biplot of Ecological Variables')
  plt.axhline(y=0, color='k', linestyle='-', alpha=0.3)
  plt.axvline(x=0, color='k', linestyle='-', alpha=0.3)
  plt.grid(True, alpha=0.3)

  plt.close(biplot_fig)
  return biplot_fig

Correlation Matrix

In [13]:
def correlation_matrix_heatmap():
  correlation_fig = plt.figure(figsize=(10, 8))
  sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
  plt.title('Correlation Matrix of Ecological Variables')

  plt.close(correlation_fig)
  return correlation_fig

Loadings Heatmap

In [14]:
def loadings_heatmap():
  loadings_heatmap = pd.DataFrame(pca.components_.T,
                              columns=[f'PC{i+1}' for i in range(X.shape[1])],
                              index=feature_names)
  loadings_fig = plt.figure(figsize=(10, 8))
  sns.heatmap(loadings_heatmap, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
  plt.title('Variable Contributions to Principal Components')

  plt.close(loadings_fig)
  return loadings_fig

Radar Chart

In [15]:
def radar_chart():
  # Number of variables
  N = len(feature_names)

  # What will be the angle of each axis in the plot (divide the plot / number of variables)
  angles = [n / float(N) * 2 * np.pi for n in range(N)]
  angles += angles[:1]  # Close the loop

  # Initialize the figure
  fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(polar=True))

  # Draw one axis per variable and add labels
  plt.xticks(angles[:-1], feature_names, size=12)

  # Draw the y axis labels (0-100%)
  ax.set_rlabel_position(0)
  plt.yticks([0.25, 0.5, 0.75, 1], ["25%", "50%", "75%", "100%"], color="grey", size=10)
  plt.ylim(0, 1)

  # Plot each site
  for i, site in enumerate(eco_data['Date']):
      values = eco_data.iloc[i, 1:].values  # Get values for this site
      # Scale to 0-1 range for each variable
      min_vals = eco_data.iloc[:, 1:].min()
      max_vals = eco_data.iloc[:, 1:].max()
      scaled_values = (values - min_vals) / (max_vals - min_vals)

      # Close the loop
      scaled_values = np.append(scaled_values, scaled_values.iloc[0])

      # Plot values
      ax.plot(angles, scaled_values, linewidth=2, linestyle='solid', label=site)
      ax.fill(angles, scaled_values, alpha=0.1)

  # Add legend
  ax.legend(loc='upper right', bbox_to_anchor=(0.1, 0.1))
  plt.title('Ecological Characteristics by Date', size=15, y=1.1)

  plt.close(fig)
  return fig

PC1, PC2 and PC3 on one Plot

In [16]:
def pca_3d():
  fig = plt.figure(figsize=(12, 10))
  ax = fig.add_subplot(111, projection='3d')

  # Plot the observations
  ax.scatter(pca_result[:, 0], pca_result[:, 1], pca_result[:, 2], s=100, alpha=0.7)

  # Add site labels
  for i, site in enumerate(eco_data_dates):
      ax.text(pca_result[i, 0], pca_result[i, 1], pca_result[i, 2], site)

  ax.set_xlabel(f'PC1 ({explained_variance[0]:.2f}%)')
  ax.set_ylabel(f'PC2 ({explained_variance[1]:.2f}%)')
  ax.set_zlabel(f'PC3 ({explained_variance[2]:.2f}%)')
  ax.set_title('3D PCA Plot of Ecological Data')

  plt.close(fig)
  return fig

##Kriging

Load Data

In [17]:
monitoring_df = pd.read_excel(monitoring_reports_data_file)
monitoring_df['measurement_date'] = pd.to_datetime(monitoring_df['measurement_date'], format='%d/%m/%Y')
all_measurement_types = monitoring_df['measurement_type'].unique()

In [18]:
biogis_df = pd.read_csv(biogis_data_file)
biogis_df['collection_date'] = pd.to_datetime(biogis_df['collection_date'], format='%d/%m/%Y')

Find Best Kriging Variogram Model

In [19]:
def find_best_model(x, y, values, xpoints, ypoints, models=('linear', 'power', 'gaussian', 'spherical', 'exponential')):
    best_model = None
    best_score = float('inf')
    best_ok = None
    best_z, best_ss = None, None

    for model in models:
        try:
            ok = OrdinaryKriging(
                x, y, values,
                variogram_model=model,
                coordinates_type='geographic',
                verbose=False,
                enable_plotting=False
            )
            z, ss = ok.execute('grid', xpoints, ypoints)
            # Use the sum of squared values as a crude proxy for quality
            score = np.nansum(ss)
            if score < best_score:
                best_score = score
                best_model = model
                best_z, best_ss = z, ss
                best_ok = ok

        except Exception as e:
            # print(f"Model {model} failed: {e}")
            continue

    return best_ok, best_z, best_ss

Plot Kriging

In [20]:
def plot_ordinary_kriging(kriging_longitudes, kriging_latitudes, kriging_values, measurement_type_to_krig):
  plots = []
  if len(kriging_values) >= 3:
    # Create a grid for prediction (adjust the range based on your coordinates)
    grid_lon = np.linspace(kriging_longitudes.min() - 0.05, kriging_longitudes.max() + 0.05, 100)
    grid_lat = np.linspace(kriging_latitudes.min() - 0.05, kriging_latitudes.max() + 0.05, 100)

    # Perform Ordinary Kriging
    # ok = OrdinaryKriging(
    #     kriging_longitudes,
    #     kriging_latitudes,
    #     kriging_values,
    #     variogram_model='spherical',  # You can experiment with other models
    #     verbose=False,
    #     enable_plotting=False,
    # )

    ok, z, ss = find_best_model(kriging_longitudes, kriging_latitudes, kriging_values, grid_lon, grid_lat)

    # Make predictions on the grid
    # z, ss = ok.execute('grid', grid_lon, grid_lat)

    # Plotting the results
    fig_interpolation = plt.figure(figsize=(10, 8))

    # Contour plot of the Kriged values
    contour = plt.contourf(grid_lon, grid_lat, z, levels=20, cmap='viridis')
    cbar = plt.colorbar(contour, label=f'Predicted {measurement_type_to_krig}')

    # Scatter plot of the original data points
    plt.scatter(kriging_longitudes, kriging_latitudes, c=kriging_values, cmap='viridis', edgecolor='k', s=50)
    plt.title(f'Ordinary Kriging Interpolation of {measurement_type_to_krig}')
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.colorbar(label=f'{measurement_type_to_krig} Level')
    plt.axis('equal')
    plt.grid(True)
    plt.tight_layout()
    plt.close(fig_interpolation)
    plots.append(fig_interpolation)

    # Optional: Plot the variance (estimation error)
    fig_variations = plt.figure(figsize=(10, 8))
    contour_var = plt.contourf(grid_lon, grid_lat, ss, levels=20, cmap='plasma_r')
    cbar_var = plt.colorbar(contour_var, label='Kriging Variance')
    plt.scatter(kriging_longitudes, kriging_latitudes, c='black', edgecolor='white', s=50)
    plt.title(f'Ordinary Kriging Variance for {measurement_type_to_krig}')
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.axis('equal')
    plt.grid(True)
    plt.tight_layout()
    plt.close(fig_variations)
    plots.append(fig_variations)

  return plots

Plot Kriging using Data

In [21]:
def plot_kriging_at(year, month, data_source, measurement_type):
    """
    Plots Ordinary Kriging interpolation for a given year and month,
    using a specified data source and measurement type.

    Args:
        year (int): The year of the data.
        month (int): The month of the data.
        data_source (pd.DataFrame): The DataFrame containing the data
                                     (either monitoring_df or biogis_df).
        measurement_type (str): The type of measurement to krig
                                (e.g., 'coliform_bacteria' or 'individuals').

    Returns:
        list: A list of matplotlib figure objects (interpolation and variance plots),
              or [None, None] if not enough data points are available.
    """
    if data_source is monitoring_df:
        filtered_data = data_source[
            (data_source['measurement_date'].dt.year == year) &
            (data_source['measurement_date'].dt.month == month) &
            (data_source['measurement_type'] == measurement_type)
        ]

        kriging_longitudes = []
        kriging_latitudes = []
        kriging_values = []

        for station, coords in station_coords.items():
            if station in filtered_data.columns and not pd.isna(filtered_data[station].iloc[0]):
                kriging_longitudes.append(coords[0])
                kriging_latitudes.append(coords[1])
                kriging_values.append(filtered_data[station].iloc[0])

        kriging_longitudes = np.array(kriging_longitudes)
        kriging_latitudes = np.array(kriging_latitudes)
        kriging_values = np.array(kriging_values)

    elif data_source is biogis_df:
        filtered_data = data_source[
            (data_source['collection_date'].dt.year == year) &
             (data_source['collection_date'].dt.month == month)
        ]

        aggregated_individuals = filtered_data.groupby(['longitude', 'latitude'])[measurement_type].sum().reset_index()
        kriging_longitudes = aggregated_individuals['longitude'].values
        kriging_latitudes = aggregated_individuals['latitude'].values
        kriging_values = aggregated_individuals[measurement_type].values

    else:
        return [None, None] # Unsupported data source

    plots = plot_ordinary_kriging(kriging_longitudes, kriging_latitudes, kriging_values, measurement_type)

    if not plots:
        return [None, None]
    return plots

###Intervining Variables

Read and Process Data

In [22]:
# Read meteorological data
meteo_data = pd.read_csv(meteo_data_file)

# Rename columns (from Hebrew to English)
meteo_data = meteo_data.rename(columns={
    meteo_data.columns[0]: 'station',
    meteo_data.columns[1]: 'date',
    meteo_data.columns[2]: 'temperature',
    meteo_data.columns[3]: 'wind_velocity'
})

# Convert date column to datetime and extract month and year
meteo_data['date'] = pd.to_datetime(meteo_data['date'], format="%d-%m-%Y %H:%M")
meteo_data['month'] = meteo_data['date'].dt.month
meteo_data['year'] = meteo_data['date'].dt.year
meteo_data.drop(columns=['date'], inplace=True)

In [23]:
min_year = meteo_data['year'].min()
max_year = meteo_data['year'].max()

temperatures = [meteo_data[(meteo_data['year'] == year) & (meteo_data['month'] == month)]['temperature'].max() for year in range(min_year, max_year + 1) for month in range(1, 13)]
wind_velocities = [meteo_data[(meteo_data['year'] == year) & (meteo_data['month'] == month)]['wind_velocity'].max() for year in range(min_year, max_year + 1) for month in range(1, 13)]

max_temperature_idx = meteo_data['temperature'].idxmax()
max_wind_velocity_idx = meteo_data['wind_velocity'].idxmax()

Plot Data

In [24]:
def plot_meteo_data_per_month():
  months = [f"{year}-{month:02d}" for year in range(min_year, max_year + 1) for month in range(1, 13)]
  max_temperature_month = f"{meteo_data.loc[max_temperature_idx]['year']}-{meteo_data.iloc[max_temperature_idx]['month']:02d}"
  max_wind_velocity_month = f"{meteo_data.loc[max_wind_velocity_idx]['year']}-{meteo_data.iloc[max_wind_velocity_idx]['month']:02d}"

  fig1 = plt.figure(figsize=(12, 8))
  plt.plot(months, temperatures, label='Max Temperature', marker='o')
  plt.plot(max_temperature_month, meteo_data.loc[max_temperature_idx]['temperature'], 'ro', label='Max Temperature (Point)')
  plt.xlabel('Month')
  plt.xticks(rotation=270)
  plt.ylabel('Temperature (°C)')
  plt.title('Max Temperature per Month')
  plt.grid(True)
  plt.legend()

  fig2 = plt.figure(figsize=(12, 8))
  plt.plot(months, wind_velocities, label='Max Wind Velocity', marker='o')
  plt.plot(max_wind_velocity_month, meteo_data.loc[max_wind_velocity_idx]['wind_velocity'], 'ro', label='Max Wind Velocity (Point)')
  plt.xlabel('Month')
  plt.xticks(rotation=270)
  plt.ylabel('Wind Velocity (m/s)')
  plt.title('Max Wind Velocity per Month')
  plt.grid(True)
  plt.legend()

  plt.close(fig1)
  plt.close(fig2)

  return [fig1, fig2]

###Scenarios

Base Scenario

In [25]:
def plot_kriging_base(measurement_type):
  # Determine the year and month for the latest biogis data
  latest_date = biogis_df['collection_date'].max()
  year = latest_date.year
  month = latest_date.month
  individuals_plots = plot_kriging_at(year, month, biogis_df, 'individuals')

  # Determine the year and month for the latest monitoring data
  latest_date = monitoring_df['measurement_date'].max()
  year = latest_date.year
  month = latest_date.month
  measurement_type_plots = plot_kriging_at(year, month, monitoring_df, measurement_type)

  plots = individuals_plots + measurement_type_plots
  return plots

Pressured Scenario

In [26]:
def plot_kriging_at_pressure(measurement_type):
  year = meteo_data.loc[max_wind_velocity_idx]['year']
  month = meteo_data.loc[max_wind_velocity_idx]['month']

  individuals_plots = plot_kriging_at(year, month, biogis_df, 'individuals')
  measurement_type_plots = plot_kriging_at(year, month, monitoring_df, measurement_type)

  plots = individuals_plots + measurement_type_plots
  return plots

Rehabilitaion Scenario

In [27]:
def plot_kriging_at_rehabilitation(measurement_type):
  year = meteo_data.loc[max_temperature_idx]['year']
  month = meteo_data.loc[max_temperature_idx]['month'] + 1
  if month > 12:
    month = 1
    year += 1

  individuals_plots = plot_kriging_at(year, month, biogis_df, 'individuals')
  measurement_type_plots = plot_kriging_at(year, month, monitoring_df, measurement_type)

  plots = individuals_plots + measurement_type_plots
  return plots

## Gradio Interface

In [28]:
def plot_pca(plot_type):
  if plot_type == "Scree Plot":
    return scree_plot()
  elif plot_type == "Biplot (PC1 vs PC2)":
    return biplot_pc1_pc2()
  elif plot_type == "Correlation Matrix":
    return correlation_matrix_heatmap()
  elif plot_type == "PCA Loadings Heatmap":
    return loadings_heatmap()
  elif plot_type == "Radar Chart":
    return radar_chart()
  elif plot_type == "3D PCA Plot":
    return pca_3d()
  return None

In [29]:
def plot_kriging_at_scenario(scenario, measurement_type):
  if scenario == "Base Scenario":
    return plot_kriging_base(measurement_type)
  elif scenario == "Pressure Scenario":
    return plot_kriging_at_pressure(measurement_type)
  elif scenario == "Rehabilitation Scenario":
    return plot_kriging_at_rehabilitation(measurement_type)
  return None

In [30]:
with gr.Blocks() as demo:
    with gr.Tabs():
        with gr.Tab("DataFrames"):
          with gr.Row():
            with gr.Column():
              gr.Markdown("### Ecological PCA Data")
              gr.DataFrame(eco_data)

            with gr.Column():
              gr.Markdown("### Biogis Observations")
              gr.DataFrame(biogis_df)

          with gr.Row():
            with gr.Column():
              gr.Markdown("### Monitoring Reports")
              gr.DataFrame(monitoring_df)

            with gr.Column():
              gr.Markdown("### Meteo Data")
              gr.DataFrame(meteo_data)

          with gr.Row():
            meteo_figs = plot_meteo_data_per_month()
            if meteo_figs[0]:
              with gr.Column():
                gr.Markdown("### Max Temperature per Month")
                gr.Plot(lambda: meteo_figs[0])
              with gr.Column():
                gr.Markdown("### Max Wind Velocity per Month")
                gr.Plot(lambda: meteo_figs[1])

        with gr.Tab("PCA Plots"):
          with gr.Row():
            plot_dropdown = gr.Dropdown(choices=["Scree Plot", "Biplot (PC1 vs PC2)", "Correlation Matrix", "PCA Loadings Heatmap", "Radar Chart", "3D PCA Plot"], label="Select Plot", value="Scree Plot")

          with gr.Row():
            plot_output = gr.Plot()

          plot_dropdown.change(
              fn=plot_pca,
              inputs=[plot_dropdown],
              outputs=[plot_output],
          )

          demo.load(fn=plot_pca, inputs=[plot_dropdown], outputs=[plot_output])

        with gr.Tab("Kriging Plots"):
            with gr.Row():
              kriging_plot_dropdown = gr.Dropdown(choices=["Base Scenario", "Pressure Scenario", "Rehabilitation Scenario"], label="Select Scenario", value="Base Scenario")
              measurement_type_for_kriging = gr.Dropdown(choices=list(all_measurement_types), label="Select Measurement Type", value=all_measurement_types[0])
            with gr.Row():
              create_kriging_plots_button = gr.Button("Create Kriging Plots", scale=1, min_width=0)

            gr.Markdown("### Individuals Kriging")
            with gr.Row():
              with gr.Column():
                kriging_individuals_plot1 = gr.Plot()
              with gr.Column():
                kriging_individuals_plot2 = gr.Plot()

            gr.Markdown("### Measurement Kriging")
            with gr.Row():
              with gr.Column():
                kriging_coliform_plot1 = gr.Plot()
              with gr.Column():
                kriging_coliform_plot2 = gr.Plot()

            create_kriging_plots_button.click(
                fn=plot_kriging_at_scenario,
                inputs=[kriging_plot_dropdown, measurement_type_for_kriging],
                outputs=[kriging_individuals_plot1, kriging_individuals_plot2, kriging_coliform_plot1, kriging_coliform_plot2]
            )

# App

In [31]:
demo.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a04d7d293b72a4a890.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
